# Titanic Survival Classification

Connect to Google Drive.

In [40]:
from google.colab import drive
drive.mount('/content/drive')
##dont need this chunk

ModuleNotFoundError: No module named 'google'

Import pandas, seaborn, and matplotlib.

In [ ]:
import pandas as pd
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

Read the train data.  

In [55]:
train_data = pd.read_csv("train.csv",
                     sep=",",  # delimiter
                     header=0, # header in first row
                     index_col=0 # ids in first column
                     )
train_data.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Read the test data.

In [ ]:
test_data = pd.read_csv("test.csv",
                     sep=",",  # delimiter
                     header=0, # header in first row
                     index_col=0 # ids in first column
                     )
test_data.head()

Check the train data for missingess, levels, etc.

In [ ]:
train_data.describe(include="all")

Check the test data for missingness, levels, etc.

In [ ]:
test_data.describe(include="all")

Select columns and create dummies.


In [42]:

predictors = ["Pclass",
              "Sex",  
              "Age",  
              "SibSp",  
              "Parch",  
              "Fare",  
              "Embarked"]
X_train = pd.get_dummies(train_data[predictors],
               drop_first=True)
y_train = train_data["Survived"]

X_test = pd.get_dummies(test_data[predictors],
               drop_first=True)
X_train.head()
X_train.describe()


,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S
count,891.000000,714.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,2.308642,29.699118,0.523008,0.381594,32.204208,0.647587,0.086420,0.722783
std,0.836071,14.526497,1.102743,0.806057,49.693429,0.477990,0.281141,0.447876
min,1.000000,0.420000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,20.125000,0.000000,0.000000,7.910400,0.000000,0.000000,0.000000
50%,3.000000,28.000000,0.000000,0.000000,14.454200,1.000000,0.000000,1.000000
75%,3.000000,38.000000,1.000000,0.000000,31.000000,1.000000,0.000000,1.000000
max,3.000000,80.000000,8.000000,6.000000,512.329200,1.000000,1.000000,1.000000


Replace missing values with the means from the training data.

In [43]:
train_means = X_train.mean()
#train_means

X_train = X_train.fillna(train_means)
#X_train.describe()
#X_train.shape

X_test = X_test.fillna(train_means)
#X_test.describe()

Fit logistic regression model and calculate accuracy using the train data.

In [44]:
my_lr = LogisticRegression(max_iter=400).fit(X_train, y_train)
lr_pred_train = my_lr.predict(X_train)
metrics.accuracy_score(y_train, lr_pred_train)

0.8013468013468014

Create confusion matrix.

In [45]:
metrics.confusion_matrix(y_train, lr_pred_train)

array([[474,  75],
       [102, 240]])

Make predictions using test data.

In [48]:
lr_pred_test = my_lr.predict(X_test)

Format predictions for output and write to csv.

In [53]:
lr_output = pd.DataFrame(lr_pred_test,
                      index=X_test.index,
                      columns=["Survived"])
lr_output.to_csv("_lr_pred.csv")

Fit decision tree model and calculate accuracy on the train data.

In [49]:
my_tree = DecisionTreeClassifier().fit(X_train, y_train)
tree_pred_train = my_tree.predict(X_train)
metrics.accuracy_score(y_train, tree_pred_train)

0.9820426487093153

Create confusion matrix for the train data.

In [50]:
pd.DataFrame(metrics.confusion_matrix(y_train, tree_pred_train, normalize="true"))

,0,1
0,0.996357,0.003643
1,0.040936,0.959064


Make predictions using test data.

In [51]:
tree_pred_test = my_tree.predict(X_test)

Format predictions for output and write to csv.

In [54]:
tree_output = pd.DataFrame(tree_pred_test,
                      index=X_test.index,
                      columns=["Survived"])
tree_output.to_csv("tree_pred.csv")